In [1]:
import numpy as np
import numba as nb
import scipy as sp
import math
import torch

In [2]:
def get_lmoments(x, nmom=5) -> np.ndarray:
    try:
        x = np.asarray(x, dtype=np.float64)
        n = x.shape[0]
        x.sort(axis=0)
    except ValueError:
        raise ValueError("Input data to estimate L-moments must be numeric.")

    if nmom <= 0 or nmom > 5:
        raise ValueError("Invalid number of sample L-moments")

    if n < nmom:
        raise ValueError("Insufficient length of data for specified nmoments")

    # First L-moment

    l1 = np.sum(x,axis=0) / sp.special.comb(n, 1, exact=True)

    if nmom == 1:
        return l1

    # Second L-moment

    comb1 = range(n)
    coefl2 = 0.5 / sp.special.comb(n, 2, exact=True)
    sum_xtrans = sum([(comb1[i] - comb1[n - i - 1]) * x[i] for i in range(n)])
    l2 = coefl2 * sum_xtrans

    if nmom == 2:
        return np.array([l1, l2])

    # Third L-moment

    comb3 = [sp.special.comb(i, 2, exact=True) for i in range(n)]
    coefl3 = 1.0 / 3.0 / sp.special.comb(n, 3, exact=True)
    sum_xtrans = sum(
        [
            (comb3[i] - 2 * comb1[i] * comb1[n - i - 1] + comb3[n - i - 1]) * x[i]
            for i in range(n)
        ]
    )
    l3 = coefl3 * sum_xtrans / l2

    if nmom == 3:
        return np.array([l1, l2, l3])

    # Fourth L-moment

    comb5 = [sp.special.comb(i, 3, exact=True) for i in range(n)]
    coefl4 = 0.25 / sp.special.comb(n, 4, exact=True)
    sum_xtrans = sum(
        [
            (
                comb5[i]
                - 3 * comb3[i] * comb1[n - i - 1]
                + 3 * comb1[i] * comb3[n - i - 1]
                - comb5[n - i - 1]
            )
            * x[i]
            for i in range(n)
        ]
    )
    l4 = coefl4 * sum_xtrans / l2

    if nmom == 4:
        return np.array([l1, l2, l3, l4])

    # Fifth L-moment

    comb7 = [sp.special.comb(i, 4, exact=True) for i in range(n)]
    coefl5 = 0.2 / sp.special.comb(n, 5, exact=True)
    sum_xtrans = sum(
        [
            (
                comb7[i]
                - 4 * comb5[i] * comb1[n - i - 1]
                + 6 * comb3[i] * comb3[n - i - 1]
                - 4 * comb1[i] * comb5[n - i - 1]
                + comb7[n - i - 1]
            )
            * x[i]
            for i in range(n)
        ]
    )
    l5 = coefl5 * sum_xtrans / l2

    return np.array([l1, l2, l3, l4, l5])

In [42]:
@nb.njit(parallel=True,cache=True)
def get_lmoments_jit(x:np.ndarray, nmom:int=5) -> np.ndarray:

    x = np.asarray(x, dtype=np.float64)
    n = x.shape[0]

    with nb.objmode(x='float64[:,:,:]'):
        x = np.sort(x,axis=0)
    
    sum_xtrans = np.empty_like(x[0])

    def comb(n, r):
        if r < 0 or r > n:
            return 0
        if r == 0 or r == n:
            return 1
        c = 1
        for i in range(1, r + 1):
            c = c * (n - i + 1) // i
        return c

    # First L-moment

    l1 = np.sum(x,axis=0) / comb(n, 1)

    if nmom == 1:
        return l1

    # Second L-moment

    comb1 = np.arange(n)
    coefl2 = 0.5 / comb(n, 2)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (comb1[i] - comb1[n - 1 - i]) * x[i]
    l2 = coefl2 * sum_xtrans

    if nmom == 2:
        return l2

    # Third L-moment

    comb3 = np.zeros(n)
    for i in range(n):
        comb3[i] = comb(i, 2)
    coefl3 = 1.0 / 3.0 / comb(n, 3)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (
            comb3[i] - 2 * comb1[i] * comb1[n - 1 - i] + comb3[n - 1 - i]
        ) * x[i]
    l3 = coefl3 * sum_xtrans / l2

    if nmom == 3:
        return l3

    # Fourth L-moment

    comb5 = np.zeros(n)
    for i in range(n):
        comb5[i] = comb(i, 3)
    coefl4 = 0.25 / comb(n, 4)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (
            comb5[i]
            - 3 * comb3[i] * comb1[n - 1 - i]
            + 3 * comb1[i] * comb3[n - 1 - i]
            - comb5[n - 1 - i]
        ) * x[i]
    l4 = coefl4 * sum_xtrans / l2

    if nmom == 4:
        return l4

    # Fifth L-moment

    comb7 = np.zeros(n)
    for i in range(n):
        comb7[i] = comb(i, 4)
    coefl5 = 0.2 / comb(n, 5)
    sum_xtrans[:] = 0
    for i in nb.prange(n):
        sum_xtrans += (
            comb7[i]
            - 4 * comb5[i] * comb1[n - 1 - i]
            + 6 * comb3[i] * comb3[n - 1 - i]
            - 4 * comb1[i] * comb5[n - 1 - i]
            + comb7[n - 1 - i]
        ) * x[i]

    l5 = coefl5 * sum_xtrans / l2

    return l5

In [22]:
testdata = np.random.rand(192,1000,1000)

In [23]:
get_lmoments(testdata.copy(),nmom=5)[4]

array([[ 0.01249427,  0.00118409,  0.00218354, ...,  0.00743184,
         0.00754632,  0.01475133],
       [-0.02763869,  0.00173636,  0.00074973, ...,  0.00526861,
        -0.00975529,  0.00676357],
       [-0.01350876, -0.00120716,  0.01341589, ..., -0.00107287,
        -0.00493901, -0.01149845],
       ...,
       [ 0.01836404, -0.00982705,  0.00175418, ..., -0.00675716,
         0.00569623, -0.00643706],
       [-0.00810628, -0.00389847, -0.01430677, ..., -0.01311679,
        -0.0063086 , -0.00173842],
       [-0.00242868,  0.02011434,  0.0005192 , ..., -0.00102343,
        -0.02688147, -0.00956572]])

In [44]:
get_lmoments_jit(testdata.copy(),nmom=5)

array([[ 0.01249427,  0.00118409,  0.00218354, ...,  0.00743184,
         0.00754632,  0.01475133],
       [-0.02763869,  0.00173636,  0.00074973, ...,  0.00526861,
        -0.00975529,  0.00676357],
       [-0.01350876, -0.00120716,  0.01341589, ..., -0.00107287,
        -0.00493901, -0.01149845],
       ...,
       [ 0.01836404, -0.00982705,  0.00175418, ..., -0.00675716,
         0.00569623, -0.00643706],
       [-0.00810628, -0.00389847, -0.01430677, ..., -0.01311679,
        -0.0063086 , -0.00173842],
       [-0.00242868,  0.02011434,  0.0005192 , ..., -0.00102343,
        -0.02688147, -0.00956572]])

In [47]:
%timeit get_lmoments_jit(testdata,nmom=5)

4.7 s ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
%timeit get_lmoments(testdata,nmom=5)

5.25 s ± 90.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
